In [25]:
import pandas as pd
import sqlite3 as db
import os
from bs4 import BeautifulSoup

In [26]:
# Path to the folder with your "hello_class_*.html" files
data_dir = '/Users/eliasbrown/Desktop/DATA-304/data/assignment_3/raw/downloaded_class_submissions'

# Open (or create) the sqlite database
db_path = os.path.join('/Users/eliasbrown/Desktop/DATA-304/data/assignment_3//altered', 'parsed_bios.sqlite')
conn = db.connect(db_path)
c = conn.cursor()

In [27]:
# Create the "names" table if it doesn't exist
c.execute('''
    CREATE TABLE IF NOT EXISTS names (
        name_id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT UNIQUE
    )
''')

# Create the "favorites" table if it doesn't exist
c.execute('''
    CREATE TABLE IF NOT EXISTS favorites (
        favorite_id INTEGER PRIMARY KEY AUTOINCREMENT,
        name_id INTEGER,
        category TEXT,
        favorite TEXT,
        FOREIGN KEY(name_id) REFERENCES names(name_id)
    )
''')

In [28]:
# Loop over each 'hello_class_...' file in the directory
for file_name in os.listdir(data_dir):
    if file_name.startswith('hello_class') and file_name.endswith('.html'):
        file_path = os.path.join(data_dir, file_name)
        with open(file_path, 'r', encoding='utf-8') as f:
            soup = BeautifulSoup(f, 'html.parser')

            # Get the student's name from the <h1> tag
            name_tag = soup.find('h1')
            if not name_tag:
                continue
            name_value = name_tag.get_text(strip=True)

            # Insert this name into the "names" table (if it's not already there)
            c.execute('SELECT name_id FROM names WHERE name = ?', (name_value,))
            existing = c.fetchone()
            if existing:
                # We already have this name in the table
                name_id = existing[0]
            else:
                # Insert new name
                c.execute('INSERT INTO names (name) VALUES (?)', (name_value,))
                name_id = c.lastrowid

            # Find the table with category/favorite data
            table = soup.find('table')
            if not table:
                continue

            # The first row is typically headers, so skip that row
            rows = table.find_all('tr')[1:]
            for row in rows:
                cells = row.find_all('td')
                if len(cells) < 2:
                    continue
                category = cells[0].get_text(strip=True)
                favorite = cells[1].get_text(strip=True)

                # Insert a record in the "favorites" table
                c.execute('''
                    INSERT INTO favorites (name_id, category, favorite)
                    VALUES (?, ?, ?)
                ''', (name_id, category, favorite))

In [29]:
# Loop over each 'hello_class_...' file in the directory
for file_name in os.listdir(data_dir):
    if file_name.startswith('hello_class') and file_name.endswith('.html'):
        file_path = os.path.join(data_dir, file_name)
        with open(file_path, 'r', encoding='utf-8') as f:
            soup = BeautifulSoup(f, 'html.parser')

            # Get the student's name from the <h1> tag
            name_tag = soup.find('h1')
            if not name_tag:
                continue
            name_value = name_tag.get_text(strip=True)

            # Insert this name into the "names" table (if it's not already there)
            c.execute('SELECT name_id FROM names WHERE name = ?', (name_value,))
            existing = c.fetchone()
            if existing:
                # We already have this name in the table
                name_id = existing[0]
            else:
                # Insert new name
                c.execute('INSERT INTO names (name) VALUES (?)', (name_value,))
                name_id = c.lastrowid

            # Find the table with category/favorite data
            table = soup.find('table')
            if not table:
                continue

            # The first row is typically headers, so skip that row
            rows = table.find_all('tr')[1:]
            for row in rows:
                cells = row.find_all('td')
                if len(cells) < 2:
                    continue
                category = cells[0].get_text(strip=True)
                favorite = cells[1].get_text(strip=True)

                # Insert a record in the "favorites" table
                c.execute('''
                    INSERT INTO favorites (name_id, category, favorite)
                    VALUES (?, ?, ?)
                ''', (name_id, category, favorite))

In [30]:
# Commit the changes and close the connection
conn.commit()
conn.close()